# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [177]:
problem = '''You are in charge of the supply purchasing of a company that produces two kinds of drugs. 
The drugs contain a specific active agent, which is extracted from two different kinds of raw materials that should be purchased on the market. 

The goal is to maximize the total profit obtained from producing the drugs, which means minimizing purchasing costs for buying the raw materials and minimizing operational costs for producing the drugs while maximizing sales. 

You are given a budget for purchasing raw materials and operating the production process of the drugs which cannot be exceeded. 
Additionally, you need to keep the capacity constraints for the production of the drugs in mind as there is only a limited amount of manpower as well as equipment hours available. 
Your company can also only store a limited amount of raw materials. 
Most importantly, the amount of active ingredient extracted from the raw materials you purchase needs to satisfy the required amount of active ingredient in the drugs your company produces. 
Importantly, your experience with the production of the drugs has shown the the amount of active ingredient your process extracts from the two raw materials can vary by up to 0.5% and 2%, respectively. 
During purchasing, you need to make sure that no matter how much the amount of active ingredient varies, the required amount for the drug production is always met. '''

## 2. Generate the mathematical model

In [187]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [188]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Drug Production

**Parameters:**

* **Raw Materials:**
    * $R$: Set of raw materials, indexed by $r$ 
    * $C_r$: Cost per unit of raw material $r$ 
    * $S_r$: Available supply of raw material $r$ in the market
    * $A_r$: Nominal amount of active ingredient extracted per unit of raw material $r$
    * $V_r$: Variability in the amount of active ingredient extracted from raw material $r$ (e.g., 0.005 for 0.5% variability)
    * $M_r$: Maximum storable amount of raw material $r$
* **Drugs:**
    * $D$: Set of drugs, indexed by $d$
    * $P_d$: Selling price per unit of drug $d$
    * $Q_d$: Required amount of active ingredient per unit of drug $d$
    * $O_d$: Operational cost per unit of drug $d$
    * $H_d$: Equipment hours required per unit of drug $d$
    * $L_d$: Manpower required per unit of drug $d$
* **Production:**
    * $B$: Total budget for purchasing raw materials and operating the production process
    * $E$: Total available equipment hours
    * $W$: Total available manpower

**Decision Variables:**

* $X_{r}$: Amount of raw material $r$ to purchase.
* $Y_d$: Amount of drug $d$ to produce.

**Objective Function:**

Maximize total profit:

$$ \sum_{d \in D} P_d Y_d - \sum_{r \in R} C_r X_r - \sum_{d \in D} O_d Y_d$$

**Constraints:**

* **Budget Constraint:**
 $$ \sum_{r \in R} C_r X_r + \sum_{d \in D} O_d Y_d \le B $$
* **Raw Material Supply Constraint:**
 $$ X_r \le S_r,  \forall r \in R $$
* **Storage Capacity Constraint:**
 $$ X_r \le M_r,  \forall r \in R $$
* **Active Ingredient Requirement Constraint:** 
    * For each drug $d$, the total amount of active ingredient extracted from the raw materials, considering the variability, must meet the required amount:
    $$ \sum_{r \in R} (A_r - V_r) X_r \ge \sum_{d \in D} Q_d Y_d, \forall d \in D $$
* **Equipment Hours Constraint:**
 $$ \sum_{d \in D} H_d Y_d \le E $$
* **Manpower Constraint:**
 $$ \sum_{d \in D} L_d Y_d \le W $$
* **Non-Negativity Constraint:**
 $$ X_r \ge 0, \forall r \in R $$
 $$ Y_d \ge 0, \forall d \in D $$

This model represents the optimization problem to maximize profit while considering budget, resource limitations, and variations in active ingredient extraction. By solving this model, you can determine the optimal purchasing and production plan for your company. 


## 3. Generate the pyomo code

In [189]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [190]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with your actual data
raw_materials = ['RM1', 'RM2', 'RM3']
drugs = ['DrugA', 'DrugB']

cost_rm = {'RM1': 10, 'RM2': 15, 'RM3': 12}
supply_rm = {'RM1': 1000, 'RM2': 1500, 'RM3': 800}
active_ingredient_amount = {'RM1': 0.1, 'RM2': 0.15, 'RM3': 0.08}
variability_rm = {'RM1': 0.005, 'RM2': 0.01, 'RM3': 0.008}
max_storage = {'RM1': 500, 'RM2': 700, 'RM3': 400}

selling_price = {'DrugA': 50, 'DrugB': 60}
active_ingredient_required = {'DrugA': 0.05, 'DrugB': 0.08}
operational_cost = {'DrugA': 5, 'DrugB': 8}
equipment_hours = {'DrugA': 2, 'DrugB': 3}
manpower = {'DrugA': 1, 'DrugB': 2}

total_budget = 100000
total_equipment_hours = 4000
total_manpower = 2500

# Model
model = pyo.ConcreteModel()

# Sets
model.R = pyo.Set(initialize=raw_materials)
model.D = pyo.Set(initialize=drugs)

# Parameters
model.C = pyo.Param(model.R, initialize=cost_rm)
model.S = pyo.Param(model.R, initialize=supply_rm)
model.A = pyo.Param(model.R, initialize=active_ingredient_amount)
model.V = pyo.Param(model.R, initialize=variability_rm)
model.M = pyo.Param(model.R, initialize=max_storage)

model.P = pyo.Param(model.D, initialize=selling_price)
model.Q = pyo.Param(model.D, initialize=active_ingredient_required)
model.O = pyo.Param(model.D, initialize=operational_cost)
model.H = pyo.Param(model.D, initialize=equipment_hours)
model.L = pyo.Param(model.D, initialize=manpower)

model.B = pyo.Param(initialize=total_budget)
model.E = pyo.Param(initialize=total_equipment_hours)
model.W = pyo.Param(initialize=total_manpower)

# Decision Variables
model.X = pyo.Var(model.R, domain=pyo.NonNegativeReals)
model.Y = pyo.Var(model.D, domain=pyo.NonNegativeReals)

# Objective Function
def total_profit(model):
    return sum(model.P[d] * model.Y[d] for d in model.D) - sum(model.C[r] * model.X[r] for r in model.R) - sum(model.O[d] * model.Y[d] for d in model.D) 
model.profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

# Constraints
def budget_constraint(model):
    return sum(model.C[r] * model.X[r] for r in model.R) + sum(model.O[d] * model.Y[d] for d in model.D) <= model.B
model.budget_constraint = pyo.Constraint(rule=budget_constraint)

def supply_constraint(model, r):
    return model.X[r] <= model.S[r]
model.supply_constraint = pyo.Constraint(model.R, rule=supply_constraint)

def storage_constraint(model, r):
    return model.X[r] <= model.M[r]
model.storage_constraint = pyo.Constraint(model.R, rule=storage_constraint)

def active_ingredient_constraint(model, d):
    return sum((model.A[r] - model.V[r]) * model.X[r] for r in model.R) >= sum(model.Q[d] * model.Y[d] for d in model.D)
model.active_ingredient_constraint = pyo.Constraint(model.D, rule=active_ingredient_constraint)

def equipment_hours_constraint(model):
    return sum(model.H[d] * model.Y[d] for d in model.D) <= model.E
model.equipment_hours_constraint = pyo.Constraint(rule=equipment_hours_constraint)

def manpower_constraint(model):
    return sum(model.L[d] * model.Y[d] for d in model.D) <= model.W
model.manpower_constraint = pyo.Constraint(rule=manpower_constraint)

# Solve
solver = pyo.SolverFactory('glpk') # You can choose a different solver if you prefer
results = solver.solve(model)

# Display Results
print(results)
print("Profit = ", pyo.value(model.profit))
print("Purchase:")
for r in model.R:
    print(f"  {r}: {pyo.value(model.X[r])}")
print("Production:")
for d in model.D:
    print(f"  {d}: {pyo.value(model.Y[d])}")
```

This code first defines sample data for the parameters. **You should replace this sample data with your actual data.** The rest of the code constructs the Pyomo model based on your provided mathematical formulation. The model is then solved using the GLPK solver, and finally, the results are printed. You can choose a different solver if you have it available and prefer it over GLPK. 


## 4. Input problem data and try running the generated code

In [204]:
import pyomo.environ as pyo


# # Sample Data - Replace with actual data
# raw_materials = {'RM1', 'RM2'}
# drugs = {'DrugA', 'DrugB'}

# price_rm = {'RM1': 100, 'RM2': 199.90}  
# capacity_rm = {'RM1': 500, 'RM2': 500} 
# extraction_rate = {'RM1': 0.01, 'RM2': 0.02}
# deviation_extraction = {'RM1': 0.005, 'RM2': 0.02}

# selling_price = {'DrugA': 6200, 'DrugB': 6900}
# active_ingredient_req = {'DrugA': 0.5, 'DrugB': 0.6}
# labor_hours = {'DrugA': 90, 'DrugB': 100}
# machine_hours = {'DrugA': 40, 'DrugB': 50}

# budget = 100000
# total_labor_hours = 2000
# total_machine_hours = 800

###
# Sample Data - Replace with your actual data
raw_materials = ['RM1', 'RM2']
drugs = ['DrugA', 'DrugB']

cost_rm = {'RM1': 100, 'RM2': 199.90}
supply_rm = {'RM1': 999999999, 'RM2': 9999999999999}
active_ingredient_amount = {'RM1': 0.01, 'RM2': 0.02}
variability_rm = {'RM1': 0.005, 'RM2': 0.02}
max_storage = {'RM1': 500, 'RM2': 500}

selling_price = {'DrugA': 6200, 'DrugB': 6900}
active_ingredient_required = {'DrugA': 0.5, 'DrugB': 0.6}
operational_cost = {'DrugA': 700, 'DrugB': 800}
manpower = {'DrugA': 40, 'DrugB': 50}
equipment_hours = {'DrugA': 90, 'DrugB': 100}

total_budget = 100000
total_equipment_hours = 2000
total_manpower = 800

# Model
model = pyo.ConcreteModel()

# Sets
model.R = pyo.Set(initialize=raw_materials)
model.D = pyo.Set(initialize=drugs)

# Parameters
model.C = pyo.Param(model.R, initialize=cost_rm)
model.S = pyo.Param(model.R, initialize=supply_rm)
model.A = pyo.Param(model.R, initialize=active_ingredient_amount)
model.V = pyo.Param(model.R, initialize=variability_rm)
model.M = pyo.Param(model.R, initialize=max_storage)

model.P = pyo.Param(model.D, initialize=selling_price)
model.Q = pyo.Param(model.D, initialize=active_ingredient_required)
model.O = pyo.Param(model.D, initialize=operational_cost)
model.H = pyo.Param(model.D, initialize=equipment_hours)
model.L = pyo.Param(model.D, initialize=manpower)

model.B = pyo.Param(initialize=total_budget)
model.E = pyo.Param(initialize=total_equipment_hours)
model.W = pyo.Param(initialize=total_manpower)

# Decision Variables
model.X = pyo.Var(model.R, domain=pyo.NonNegativeReals)
model.Y = pyo.Var(model.D, domain=pyo.NonNegativeReals)

# Objective Function
def total_profit(model):
    return sum(model.P[d] * model.Y[d] for d in model.D) - sum(model.C[r] * model.X[r] for r in model.R) - sum(model.O[d] * model.Y[d] for d in model.D) 
model.profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

# Constraints
def budget_constraint(model):
    return sum(model.C[r] * model.X[r] for r in model.R) + sum(model.O[d] * model.Y[d] for d in model.D) <= model.B
model.budget_constraint = pyo.Constraint(rule=budget_constraint)

def supply_constraint(model, r):
    return model.X[r] <= model.S[r]
model.supply_constraint = pyo.Constraint(model.R, rule=supply_constraint)

def storage_constraint(model, r):
    return model.X[r] <= model.M[r]
model.storage_constraint = pyo.Constraint(model.R, rule=storage_constraint)

def active_ingredient_constraint(model, d):
    return sum((model.A[r] - model.V[r]) * model.X[r] for r in model.R) >= sum(model.Q[d] * model.Y[d] for d in model.D)
model.active_ingredient_constraint = pyo.Constraint(model.D, rule=active_ingredient_constraint)

def equipment_hours_constraint(model):
    return sum(model.H[d] * model.Y[d] for d in model.D) <= model.E
model.equipment_hours_constraint = pyo.Constraint(rule=equipment_hours_constraint)

def manpower_constraint(model):
    return sum(model.L[d] * model.Y[d] for d in model.D) <= model.W
model.manpower_constraint = pyo.Constraint(rule=manpower_constraint)

# Solve
solver = pyo.SolverFactory('glpk') # You can choose a different solver if you prefer
results = solver.solve(model)

# Display Results
print(results)
print("Profit = ", pyo.value(model.profit))
print("Purchase:")
for r in model.R:
    print(f"  {r}: {pyo.value(model.X[r])}")
print("Production:")
for d in model.D:
    print(f"  {d}: {pyo.value(model.Y[d])}")


Problem: 
- Name: unknown
  Lower bound: 0.0
  Upper bound: 0.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 4
  Number of nonzeros: 18
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.03930521011352539
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Profit =  0.0
Purchase:
  RM1: 0.0
  RM2: 0.0
Production:
  DrugA: 0.0
  DrugB: 0.0


## 5. Correct the code to verify model viability (optional)

In [205]:
import pyomo.environ as pyo

# Parameters
p1, p2 = 6200, 6900  # Prices of drugs
c1, c2, c3, c4 = 100, 199.90, 700, 800 # Costs
a1, a2, a3, a4 = 0.01, 0.02, 0.500, 0.600  # Active ingredient extraction coefficients
t1M, t2M, t1E, t2E = 90.0, 100.0, 40.0, 50.0  # Time requirements
MPH, EH = 2000, 800  # Manpower and equipment hours limits
S, B = 1000, 100000  # Storage and budget limits

# Create a model instance
model = pyo.ConcreteModel()

# Declare decision variables
model.d1 = pyo.Var(domain=pyo.NonNegativeReals)
model.d2 = pyo.Var(domain=pyo.NonNegativeReals)
model.r1 = pyo.Var(domain=pyo.NonNegativeReals)
model.r2 = pyo.Var(domain=pyo.NonNegativeReals)

# Objective Function
model.obj = pyo.Objective(expr=(p1*model.d1 + p2*model.d2 - 
                                (c1*model.r1 + c2*model.r2 + c3*model.d1 + c4*model.d2)), sense=pyo.maximize)

# Constraints
model.active_ingredient = pyo.Constraint(expr=(0.995*a1*model.r1 + 0.98*a2*model.r2 - a3*model.d1 - a4*model.d2 >= 0))
model.manpower = pyo.Constraint(expr=(t1M*model.d1 + t2M*model.d2 <= MPH))
model.equipment = pyo.Constraint(expr=(t1E*model.d1 + t2E*model.d2 <= EH))
model.storage = pyo.Constraint(expr=(model.r1 + model.r2 <= S))
model.budget = pyo.Constraint(expr=(c1*model.r1 + c2*model.r2 + c3*model.d1 + c4*model.d2 <= B))

# Solve the model
solver = pyo.SolverFactory('glpk')
result = solver.solve(model)

# Display the results
print(f"Status: {result.solver.status}")
print(f"Objective value: {pyo.value(model.obj)}")
print(f"d1 = {pyo.value(model.d1)}")
print(f"d2 = {pyo.value(model.d2)}")
print(f"r1 = {pyo.value(model.r1)}")
print(f"r2 = {pyo.value(model.r2)}")
     

Status: ok
Objective value: 8294.566839287349
d1 = 17.4668656192399
d2 = 0.0
r1 = 877.731940665321
r2 = 0.0


In [206]:
model.pprint()

4 Var Declarations
    d1 : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 17.4668656192399 :  None : False : False : NonNegativeReals
    d2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    r1 : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 877.731940665321 :  None : False : False : NonNegativeReals
    r2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 6200*d1 + 6900*d2 - (100*r1 + 199.9*r2 + 700*d1 + 800*d2)

5 Constraint Declarations
    active_ingredient : Size=1, Index=None, Active=True

## 6. Printing the outputs as strings, so they can be saved.

In [207]:
print(response.text)

## Mathematical Optimization Model for Drug Production

**Parameters:**

* **Raw Materials:**
    * $R$: Set of raw materials, indexed by $r$ 
    * $C_r$: Cost per unit of raw material $r$ 
    * $S_r$: Available supply of raw material $r$ in the market
    * $A_r$: Nominal amount of active ingredient extracted per unit of raw material $r$
    * $V_r$: Variability in the amount of active ingredient extracted from raw material $r$ (e.g., 0.005 for 0.5% variability)
    * $M_r$: Maximum storable amount of raw material $r$
* **Drugs:**
    * $D$: Set of drugs, indexed by $d$
    * $P_d$: Selling price per unit of drug $d$
    * $Q_d$: Required amount of active ingredient per unit of drug $d$
    * $O_d$: Operational cost per unit of drug $d$
    * $H_d$: Equipment hours required per unit of drug $d$
    * $L_d$: Manpower required per unit of drug $d$
* **Production:**
    * $B$: Total budget for purchasing raw materials and operating the production process
    * $E$: Total available equi

In [208]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with your actual data
raw_materials = ['RM1', 'RM2', 'RM3']
drugs = ['DrugA', 'DrugB']

cost_rm = {'RM1': 10, 'RM2': 15, 'RM3': 12}
supply_rm = {'RM1': 1000, 'RM2': 1500, 'RM3': 800}
active_ingredient_amount = {'RM1': 0.1, 'RM2': 0.15, 'RM3': 0.08}
variability_rm = {'RM1': 0.005, 'RM2': 0.01, 'RM3': 0.008}
max_storage = {'RM1': 500, 'RM2': 700, 'RM3': 400}

selling_price = {'DrugA': 50, 'DrugB': 60}
active_ingredient_required = {'DrugA': 0.05, 'DrugB': 0.08}
operational_cost = {'DrugA': 5, 'DrugB': 8}
equipment_hours = {'DrugA': 2, 'DrugB': 3}
manpower = {'DrugA': 1, 'DrugB': 2}

total_budget = 100000
total_equipment_hours = 4000
total_manpower = 2500

# Model
model = pyo.ConcreteModel()

# Sets
model.R = pyo.Set(initialize=raw_materials)
model.D = pyo.Set(initialize=drugs)

# Parameters
model.C = pyo.Param(model.R, initialize=cost_rm)
model.S = pyo.Param(model.R, initialize=supply_rm)
model.A = pyo.Param(model.R, initializ